In [6]:
import pandas as pd
import numpy as np

import math

At first I just tried to go through the primes and simply increment the number to "append" (i.e., shift by correct number of digits and add). This did okay but was far too slow, especially for larger numbers. There is a nice mathematical speedup here, if you do some modular arithmetic.

We know that since $p_1 \geq 5$, then $p_2 \geq 7$ meaning that for any $p_2$, $\gcd(p_2, 10) = 1$ and obviously $\gcd(p_2, p_1) = 1$. Suppose $n = 10^{\lceil \log_{10} (p_1) \rceil}\cdot x + p_1$ (i.e., $x$ is the remaining digits after chopping off the prime at the end) is divisble by $p_2$. 

Then
$$
10^{\lceil \log_{10} (p_1) \rceil} \cdot x + p_1 \equiv 0 \mod p_2 \implies x \equiv -10^{-\lceil \log_{10} (p_1) \rceil} p_1 \mod p_2
$$
using the multiplicative inverse $10^{-\lceil \log_{10} (p_1) \rceil} \mod p_2$. We can solve for the multiplicative inverse quickly using the [extended Euclidean algorithm][1] which is like the normal Euclidean algorithm but also comes up with Bezout coefficients so that if $x \equiv a^{-1} \mod n$ then $ax + ny = 1$, and we get back $x$ and $y$.

[1]: https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm

In [25]:
def sieve(n):
    arr = [0,0,1] + [1,0]*(n//2 + 1)
    i = 3
    while i*i <= n:
        if arr[i]:  
            arr[i*i::2*i] = [0]*len(arr[i*i::2*i])
        i += 2

    ret = []
    for (i, p) in enumerate(arr):
        if p:
            ret.append(i)

    return arr, ret

pbs, ps = sieve(10**6)

In [55]:
def extended_euclidean(a, b):
    '''
    Modified extended euclidean algorithm to return multiplicative inverse.
    '''
    s1, s2, r1, r2 = 1, 0, a, b

    while r2 != 0:
        q = r1 // r2
        s1, s2, r1, r2 = s2, s1 - q*s2, r2, r1 - q*r2

    return s1 % b

In [61]:
ind = 2
ss = {}
total_sum = 0
while 5 <= ps[ind] <= 10**6:
    p1 = ps[ind]
    p2 = ps[ind+1]

    shift = pow(10, math.ceil(math.log10(p1)))
    inv = extended_euclidean(shift, p2)
    n = shift*((-p1*inv) % p2) + p1
    
    ss[(p1,p2)] = n
    total_sum += n

    ind += 1

print(total_sum)

18613426663617118
